<a href="https://colab.research.google.com/github/pachterlab/kb_docs/blob/main/Notebooks/Figure_2/Figure_2c/2_align_ebov_mutations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Align mutated ZEBOV sequences using DIAMOND (translated alignment), Kraken2 (translated alignment), kallisto (translated search), and kallisto (standard workflow)

In [1]:
import glob
# Define number of threads to use for alignments
threads = 20 # Change to 2 if not using a TPU runtime

In [2]:
# Download and define folder with mutated ZEBOV sequences
!git clone https://github.com/pachterlab/LSCHWCP_2023.git
out = "/content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/mutated_files"

Cloning into 'LSCHWCP_2023'...
remote: Enumerating objects: 3245, done.
remote: Counting objects: 100% (1727/1727), done.
remote: Compressing objects: 100% (1083/1083), done.
remote: Total 3245 (delta 1129), reused 1060 (delta 642), pack-reused 1518 (from 1)
Receiving objects: 100% (3245/3245), 288.47 MiB | 41.35 MiB/s, done.
Resolving deltas: 100% (1747/1747), done.
Updating files: 100% (409/409), done.


# Align using DIAMOND (translated alignment)
NEW: The DIAMOND alignment was added during Nat Biotech reviews.

In [3]:
# Install gget (from dev branch until release)
!pip install -q mysql-connector-python
!pip install -q git+https://github.com/pachterlab/gget.git@dev
import gget

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
# ebola_rdrp_aa.fasta contains the Ebola RdRP amino acid sequence from PalmDB (u55137)
ebov_aa_fasta = "/content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/ebola_rdrp_aa.fasta"

In [7]:
# Align unmutated sequences
orig_file = "/content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/SRR12698539_2_extracted_u10.fa"

diamond_res_orig = gget.diamond(orig_file, ebov_aa_fasta, translated=True)

# Save results
!mkdir -p diamond_alignment
diamond_res_orig.to_csv("diamond_alignment/unmutated_diamond.csv", index=False)

INFO:gget.utils:Aligning nucleotide query to amino acid reference (blastx mode).
INFO:gget.utils:Creating DIAMOND database and initiating alignment...
INFO:gget.utils:DIAMOND alignment complete.


In [9]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    # Translated alignment
    temp_df = gget.diamond(
        mut_file,
        ebov_aa_fasta,
        translated=True,
        threads=threads,
        verbose=False
        )

    # Save results
    temp_df.to_csv("diamond_alignment/" + mut_file.split("/")[-1].split(".fa")[0] + "_diamond.csv", index=False)

# Align using Kraken2 (translated alignment)

In [ ]:
# Install Kraken2 v1.0.2 (defining version for reproducibility)
! git clone https://github.com/DerrickWood/kraken2.git --branch v2.1.2
!cd kraken2 && ./install_kraken2.sh ./

Cloning into 'kraken2'...
remote: Enumerating objects: 1064, done.
remote: Counting objects: 100% (354/354), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 1064 (delta 294), reused 280 (delta 279), pack-reused 710
Receiving objects: 100% (1064/1064), 444.61 KiB | 3.47 MiB/s, done.
Resolving deltas: 100% (778/778), done.
Note: switching to '84b2874e0ba5ffc9abaebe630433a430cd0f69f4'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

make: Entering directory '/content/kraken2/src'
g++ -fopenmp -Wall -std=c++1

Create ebov amino acid ref with NCBI taxID and align with Kraken translated search:

In [ ]:
kraken2 = "/content/kraken2/kraken2"
kraken2_build = "/content/kraken2/kraken2-build"

In [ ]:
%%time
!$kraken2_build --db kraken_ebov_protein --protein --download-taxonomy

Downloaded accession to taxon map(s)
Uncompressing taxonomy data... done.
Untarring taxonomy tree data... done.


In [ ]:
# ebola_rdrp_aa.fasta contains the Ebola RdRP amino acid sequence from PalmDB (u55137) labeled with the NCBI Ebola taxonomy ID (186538)
ebov_aa_fasta = "/content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/ebola_rdrp_aa.fasta"

In [ ]:
# Add amino acid ebola sequence to Kraken db
!$kraken2_build \
    --db kraken_ebov_protein \
    --protein \
    --no-masking \
    --add-to-library $ebov_aa_fasta

Added "/content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/ebola_rdrp_aa.fasta" to library (kraken_ebov_protein)


Note: Also tried 31mers (instead of default 15mers) which led to no alignments.

In [ ]:
# Build index
!$kraken2_build \
        --protein \
        --db kraken_ebov_protein \
        --build \
        --threads $threads

Creating sequence ID to taxonomy ID map (step 1)...
Sequence ID to taxonomy ID map complete. [0.018s]
Estimating required capacity (step 2)...
Estimated hash table requirement: 48272 bytes
Capacity estimation complete. [0.020s]
Building database files (step 3)...
Taxonomy parsed and converted.
CHT created with 4 bits reserved for taxid.
Completed processing of 1 sequences, 109 aa
Writing data to disk...  complete.
Database files completed. [6.717s]
Database construction complete. [Total: 6.778s]


Align with Kraken2 (translated search):

In [ ]:
protein_krakendb = "kraken_ebov_protein"
protein_kraken_out = "kraken2_alignment/translated"
!mkdir -p $protein_kraken_out

In [ ]:
# Align unmutated sequences
orig_file = "/content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/SRR12698539_2_extracted_u10.fa"
out_file1 = "0_ebov_snp_0_ms.k2report"
out_file2 = "0_ebov_snp_0_ms.kraken2"

!$kraken2 \
      --db $protein_krakendb \
      --threads $threads \
      --report-minimizer-data \
      --report $protein_kraken_out/$out_file1 \
      $orig_file > $protein_kraken_out/$out_file2

Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.007s (5534.2 Kseq/m, 487.01 Mbp/m).
  636 sequences classified (94.08%)
  40 sequences unclassified (5.92%)


In [ ]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    out_file1 = mut_file.split("/")[-1].split(".fa")[0] + ".k2report"
    out_file2 = mut_file.split("/")[-1].split(".fa")[0] + ".kraken2"

    !$kraken2 \
            --db $protein_krakendb \
            --threads $threads \
            --report-minimizer-data \
            --report $protein_kraken_out/$out_file1 \
            $mut_file > $protein_kraken_out/$out_file2

Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.009s (4597.1 Kseq/m, 404.54 Mbp/m).
  230 sequences classified (34.02%)
  446 sequences unclassified (65.98%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.007s (6230.4 Kseq/m, 548.28 Mbp/m).
  0 sequences classified (0.00%)
  676 sequences unclassified (100.00%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.008s (5136.1 Kseq/m, 451.98 Mbp/m).
  0 sequences classified (0.00%)
  676 sequences unclassified (100.00%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.009s (4638.1 Kseq/m, 408.15 Mbp/m).
  0 sequences classified (0.00%)
  676 sequences unclassified (100.00%)
Loading database information... done.
676 sequences (0.06 Mbp) processed in 0.009s (4381.1 Kseq/m, 385.53 Mbp/m).
  515 sequences classified (76.18%)
  161 sequences unclassified (23.82%)
Loading database information... done.
676 sequences (0.06 Mbp) pro

# Align using kallisto bustools (translated alignment)

In [ ]:
# Install kallisto from source
!git clone https://github.com/pachterlab/kallisto.git
!cd kallisto && mkdir build && cd build && cmake .. && make

Cloning into 'kallisto'...
remote: Enumerating objects: 8080, done.
remote: Counting objects: 100% (1162/1162), done.
remote: Compressing objects: 100% (313/313), done.
remote: Total 8080 (delta 880), reused 985 (delta 847), pack-reused 6918
Receiving objects: 100% (8080/8080), 9.11 MiB | 6.93 MiB/s, done.
Resolving deltas: 100% (5306/5306), done.
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting 

In [ ]:
# Install bustools from source
!git clone https://github.com/BUStools/bustools.git
!cd bustools && mkdir build && cd build && cmake .. && make

Cloning into 'bustools'...
remote: Enumerating objects: 3582, done.
remote: Counting objects: 100% (566/566), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 3582 (delta 449), reused 360 (delta 360), pack-reused 3016
Receiving objects: 100% (3582/3582), 3.42 MiB | 13.41 MiB/s, done.
Resolving deltas: 100% (1467/1467), done.
CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting C

In [ ]:
# Define paths to kallisto and bustools binaries
kallisto = "/content/kallisto/build/src/kallisto"
bustools = "/content/bustools/build/src/bustools"

In [ ]:
# Get PalmDB reference files
virus_t2g = "/content/LSCHWCP_2023/PalmDB/palmdb_clustered_t2g.txt"
virus_fasta = "/content/LSCHWCP_2023/PalmDB/palmdb_rdrp_seqs.fa"

Generate reference index:

In [ ]:
# Download dog and macaque transcriptome and genome (defining Ensembl version (110) here for reproducibility)
!pip install -q gget
!gget ref -w dna,cdna -r 110 -d canis_lupus_familiaris
!gget ref -w dna,cdna -r 110 -d macaca_mulatta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 55.1 MB/s eta 0:00:00
Fri Dec  8 18:45:32 2023 INFO Fetching reference information for canis_lupus_familiaris from Ensembl release: 110.
{
    "canis_lupus_familiaris": {
        "genome_dna": {
            "ftp": "http://ftp.ensembl.org/pub/release-110/fasta/canis_lupus_familiaris/dna/Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz",
            "ensembl_release": 110,
            "release_date": "2023-04-21",
            "release_time": "17:46",
            "bytes": "693M"
        },
        "transcriptome_cdna": {
            "ftp": "http://ftp.ensembl.org/pub/release-110/fasta/canis_lupus_familiaris/cdna/Canis_lupus_familiaris.ROS_Cfam_1.0.cdna.all.fa.gz",
            "ensembl_release": 110,
            "release_date": "2023-04-22",
            "release_time": "16:19",
            "bytes": "26M"
        }
    }
}
  % Total    % Received 

In [ ]:
# Concatenate transcriptomes and genomes into one file
canine_macaque_fasta = "macaque_dog.cdna_dna.fa.gz"
!cat \
  Macaca_mulatta.Mmul_10.cdna.all.fa.gz \
  Macaca_mulatta.Mmul_10.dna.toplevel.fa.gz \
  Canis_lupus_familiaris.ROS_Cfam_1.0.cdna.all.fa.gz \
  Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz \
  > $canine_macaque_fasta

In [ ]:
%%time
# Generate PalmDB reference index palmdb_rdrp_seqs with macaque and dog (MDCK)
virus_index = "palmdb_macaque_dog_dlist_cdna_dna.idx"

!$kallisto index \
    -t $threads \
    --aa \
    --d-list=$canine_macaque_fasta \
    -i $virus_index \
    $virus_fasta


[index] --d-list-overhang was set to 3 (with --aa, the d-list overhang must be >= 3)
[build] loading fasta file /content/LSCHWCP_2023/PalmDB/palmdb_rdrp_seqs.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508965 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unit

Align with kallisto translated search:

In [ ]:
# Define folder to save kallisto alignments
kallisto_out = "kallisto_alignment/translated"
!mkdir -p $kallisto_out

In [ ]:
# Align unmutated sequences
mut_file = orig_file
mut_name = "0_ebov_snp_0_ms"

!mkdir -p $kallisto_out/$mut_name

!$kallisto bus \
    -i $virus_index \
    -o $kallisto_out/$mut_name \
    --aa \
    -x bulk \
    -t $threads \
    $mut_file

!$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

!$bustools count \
    --genecounts \
    --cm \
    -o $kallisto_out/$mut_name/bustools_count/ \
    -g $virus_t2g \
    -e $kallisto_out/$mut_name/matrix.ec \
    -t $kallisto_out/$mut_name/transcripts.txt \
    $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,757
[index] number of D-list k-mers: 90,815
[quant] running in single-end mode
[quant] will process file 1: /content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/SRR12698539_2_extracted_u10.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 676 reads pseudoaligned

 all fits in buffer
Read in 676 BUS records
reading time 1.2e-05s
sorting time 6.7e-05s
writing time 7.1e-05s


In [ ]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    mut_name = mut_file.split("/")[-1].split(".fa")[0]

    !mkdir -p $kallisto_out/$mut_name

    !$kallisto bus \
        -i $virus_index \
        -o $kallisto_out/$mut_name \
        --aa \
        -x bulk \
        -t $threads \
        $mut_file

    !$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

    !$bustools count \
        --genecounts \
        --cm \
        -o $kallisto_out/$mut_name/bustools_count/ \
        -g $virus_t2g \
        -e $kallisto_out/$mut_name/matrix.ec \
        -t $kallisto_out/$mut_name/transcripts.txt \
        $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,757
[index] number of D-list k-mers: 90,815
[quant] running in single-end mode
[quant] will process file 1: /content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/mutated_files/2_ebov_snp_05_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 490 reads pseudoaligned

 all fits in buffer
Read in 490 BUS records
reading time 8e-06s
sorting time 3.1e-05s
writing time 6.4e-05s

[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,757
[index] number of D-list k-mers: 90,815
[quant] running in single-end mode
[quant] will process file 1: /content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/mutated_files/5_ebov_snp_25_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 10 reads pseudoaligned

 all fits in buffer
Read in 10 BUS records
reading time 4e-06s
sorting time 7e-06s
writing time 4e-

# Align using kallisto bustools (standard workflow)

The notebook showing how the ZEBOV reference files 'ebov_GCA_000848505.idx' and 'ebov_GCA_000848505_t2g.txt' were generated can be found [here](https://github.com/pachterlab/LSCHWCP_2023/blob/main/Notebooks/ebola_ref/generate_ebov_ref.ipynb).

In [ ]:
kallisto_out = f"kallisto_alignment/standard"
!mkdir -p $kallisto_out

# Ebola reference index generated using the standard kb ref workflow
# and the GCA_000848505 Zaire ebolavirus genome assembly
ebola_t2g = "/content/LSCHWCP_2023/Notebooks/ebola_ref/ebov_GCA_000848505_t2g.txt"
ebola_index = "/content/LSCHWCP_2023/Notebooks/ebola_ref/ebov_GCA_000848505.idx"

In [ ]:
mut_file = orig_file
mut_name = "0_ebov_snp_0_ms"

!mkdir -p $kallisto_out/$mut_name

!$kallisto bus \
    -i $ebola_index \
    -o $kallisto_out/$mut_name \
    -x bulk \
    -t $threads \
    $mut_file

!$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

!$bustools count \
    --genecounts \
    --cm \
    -o $kallisto_out/$mut_name/bustools_count/ \
    -g $ebola_t2g \
    -e $kallisto_out/$mut_name/matrix.ec \
    -t $kallisto_out/$mut_name/transcripts.txt \
    $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[quant] running in single-end mode
[quant] will process file 1: /content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/SRR12698539_2_extracted_u10.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 662 reads pseudoaligned

 all fits in buffer
Read in 662 BUS records
reading time 1e-05s
sorting time 1.8e-05s
writing time 2.9e-05s


In [ ]:
for mut_file in glob.glob(f"{out}/*_ms.fa"):
    mut_name = mut_file.split("/")[-1].split(".fa")[0]

    !mkdir -p $kallisto_out/$mut_name

    !$kallisto bus \
        -i $ebola_index \
        -o $kallisto_out/$mut_name \
        -x bulk \
        -t $threads \
        $mut_file

    !$bustools sort -o $kallisto_out/$mut_name/output_sorted.bus $kallisto_out/$mut_name/output.bus

    !$bustools count \
        --genecounts \
        --cm \
        -o $kallisto_out/$mut_name/bustools_count/ \
        -g $ebola_t2g \
        -e $kallisto_out/$mut_name/matrix.ec \
        -t $kallisto_out/$mut_name/transcripts.txt \
        $kallisto_out/$mut_name/output_sorted.bus


[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[quant] running in single-end mode
[quant] will process file 1: /content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/mutated_files/2_ebov_snp_05_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 410 reads pseudoaligned

 all fits in buffer
Read in 410 BUS records
reading time 7e-06s
sorting time 1e-05s
writing time 3.5e-05s

[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[quant] running in single-end mode
[quant] will process file 1: /content/LSCHWCP_2023/Notebooks/Figure_2/Figure_2c/mutated_files/5_ebov_snp_25_ms.fa
[quant] finding pseudoalignments for all files ... done
[quant] processed 676 reads, 0 reads pseudoaligned[~warn] no reads pseudoaligned.


Read in 0 BUS records
reading time 4e-06s
sorting time 0s
writing time 0s

[index] k-mer length: 31
[index] number of targets: 38
[index] number of k-mers: 17,819
[quan

Download generated alignments:

In [ ]:
# Remove all kallisto files we do not need anymore to reduce size of alignment folders
for alignment_type in ["standard", "translated"]:
  for folder in glob.glob(f"/content/kallisto_alignment/{alignment_type}/*"):
    for file in glob.glob(f"{folder}/*.*"):
      !rm $file

In [ ]:
from google.colab import files
folders_to_download = ["kraken2_alignment", "kallisto_alignment", "diamond_alignment"]

for folder in folders_to_download:
  zipped = f"{folder}.tar.gz"
  !tar -zcvf $zipped $folder
  files.download(zipped)

kallisto_alignment/
kallisto_alignment/translated/
kallisto_alignment/translated/2_ebov_snp_03_ms/
kallisto_alignment/translated/2_ebov_snp_03_ms/bustools_count/
kallisto_alignment/translated/2_ebov_snp_03_ms/bustools_count/output.mtx
kallisto_alignment/translated/2_ebov_snp_03_ms/bustools_count/output.genes.txt
kallisto_alignment/translated/2_ebov_snp_03_ms/bustools_count/output.barcodes.txt
kallisto_alignment/translated/1_ebov_snp_22_ms/
kallisto_alignment/translated/1_ebov_snp_22_ms/bustools_count/
kallisto_alignment/translated/1_ebov_snp_22_ms/bustools_count/output.mtx
kallisto_alignment/translated/1_ebov_snp_22_ms/bustools_count/output.genes.txt
kallisto_alignment/translated/1_ebov_snp_22_ms/bustools_count/output.barcodes.txt
kallisto_alignment/translated/3_ebov_snp_09_ms/
kallisto_alignment/translated/3_ebov_snp_09_ms/bustools_count/
kallisto_alignment/translated/3_ebov_snp_09_ms/bustools_count/output.mtx
kallisto_alignment/translated/3_ebov_snp_09_ms/bustools_count/output.genes.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>